In [1]:
from wandb.keras import WandbCallback
import wandb
from tqdm import tqdm
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
import numpy as np
import tensorflow as tf
import yaml
import random
import os

from utils import *

In [2]:
trainDataRaw = loadMidiInFolder(pathes['data_dir'], 'data/trainDataRaw')
testDataRaw = loadMidiInFolder(pathes['test_dir'], 'data/testDataRaw')

In [3]:
trainData = concatMidi(trainDataRaw)
testData = concatMidi(testDataRaw)

In [4]:
# Create dictionaries for each track or collect all (now).

beat2idx = []
idx2beat = []
params['vocab_size'] = []

vocabularies = set()

for track in range(len(trainData)):
    vocabularies = vocabularies | set(trainData[track]) | set(testData[track])
for track in range(len(trainData)):
    params['vocab_size'].append(len(vocabularies))
    beat2idx.append({beat: i for i, beat in enumerate(vocabularies)})
    idx2beat.append({idx:beat for beat, idx in beat2idx[track].items()})

In [5]:
params['vocab_size']

[1499, 1499, 1499, 1499]

In [6]:
for track in range(len(trainData)):

    trainData[track] = np.array([beat2idx[track][beat] for beat in trainData[track]])
    testData[track] = np.array([beat2idx[track][beat] for beat in testData[track]])

trainData = trainData.astype(int)
testData = testData.astype(int)

In [8]:
trainDatasets = []
testDatasets = []
for track in range(len(trainData)):
    trainDatasets.append(makeDataset(trainData[track], track))
    testDatasets.append(makeDataset(testData[track], track))

In [9]:
trainDatasets[0]

<BatchDataset shapes: ((16, 16), (16, 16, 1499)), types: (tf.int64, tf.float32)>

## Single track training

In [10]:
name = 'single-track-lstm-training'
track = 0
model = makeSingleTrackModel(track)
fig = plot_model(model, to_file=name+'.png', show_layer_names=True, show_shapes=True)

In [ ]:
callbacks = []
filepath = config['path']['model_dir'] + \
    "model-"+name+"-{epoch:02d}-{loss:.4f}.hdf5"
callbacks.append(ModelCheckpoint(
    filepath=filepath,
    monitor='loss',
    mode='min',
    save_weights_only=True,
    save_best_only=True))

if config['output']['wandb']:
    wandb.init(config=params, project='lstm-singletrack-js-fake')
    callbacks.append(WandbCallback(
        log_weights=True, log_evaluation=False, validation_steps=5))

history = model.fit(trainDatasets[track], validation_data=testDatasets[track], batch_size=params['batch_size'],
                    epochs=params['epochs'], verbose=1, callbacks=callbacks)


## Multi track training

In [12]:
x_train = []
y_train = []
x_val = []
y_val = []

for track in range(params['track_size']):

    x = list(trainDatasets[track].map(lambda x, y: x))[0].numpy()
    y = list(trainDatasets[track].map(lambda x, y: y))[0].numpy()

    val_x = list(testDatasets[track].map(lambda x, y: x))[0].numpy()
    val_y = list(testDatasets[track].map(lambda x, y: y))[0].numpy()

    x_train.append(x)
    y_train.append(y)
    x_val.append(val_x)
    y_val.append(val_y)

In [13]:
x_train[0].shape, y_train[0].shape, x_val[0].shape, y_val[0].shape

((16, 16), (16, 16, 1499), (16, 16), (16, 16, 1499))

In [14]:
name = 'multi-track-lstm-training-5'
model = makeMultiTrackModel()

In [ ]:
callbacks = []
filepath = config['path']['model_dir'] + \
    "model-"+name+"-{epoch:02d}-{loss:.4f}.hdf5"
callbacks.append(ModelCheckpoint(
    filepath=filepath,
    monitor='loss',
    mode='min',
    save_weights_only=True,
    save_best_only=True))

if outputs['wandb']:
    wandb.init(config=params, project='lstm-multi-track-js-fake')
    callbacks.append(WandbCallback(
        log_weights=True, log_evaluation=False, validation_steps=5))

history = model.fit(x=x_train, y=y_train, validation_data=(x_val, y_val), batch_size=params['batch_size'],
                    epochs=params['epochs'], verbose=1, callbacks=callbacks)

if outputs['wandb']:
    wandb.finish()

## Generate Single Track

In [12]:
initBeatIndex = random.randint(0, len(trainData[0])-params['sequence_length'])
initBeat = np.array([trainData[0, initBeatIndex:initBeatIndex+params['sequence_length']]])

In [13]:
initBeat.shape

(1, 16)

In [15]:
weight = 'model-single-track-lstm-training-05-4.0770.hdf5'

model = makeSingleTrackModel(track, 1)
model.build(tf.TensorShape([1, None]))
model.load_weights(os.path.join(pathes['model_dir'], weight))
model.reset_states()
pred = model(initBeat)
pred = pred.numpy().squeeze()

music_sequence = []

for i in tqdm(range(outputs['length'])):

    topk = tf.math.top_k(pred[-1], 3)
    topkChoices = topk[1].numpy().squeeze()
    topkValues = topk[0].numpy().squeeze()

    # Apply random
    next_beat = None
    if np.random.uniform(0, 1) < .5:
        next_beat = topkChoices[0]
    else:
        p_choices = tf.math.softmax(topkValues[1:]).numpy()
        next_beat = np.random.choice(topkChoices[1:], 1, p=p_choices)[0]

    music_sequence.append(idx2beat[track][next_beat])

    pred = model(np.array([[next_beat]]))
    pred = tf.expand_dims(pred, 0)

100%|██████████| 50/50 [00:00<00:00, 70.71it/s]


In [ ]:
generateMidi(name+'.mid', [music_sequence])

## Generate Multi Track

In [ ]:
initBeatIndex = random.randint(0, len(trainData[0])-params['sequence_length'])
initBeat = []
for track in range(params['track_size']):
    initBeat.append(np.array([trainData[track, initBeatIndex:initBeatIndex+params['sequence_length']]]))

In [ ]:
weight = 'model-multi-track-lstm-training-4-998-1.0914.hdf5'

In [ ]:
model = makeMultiTrackModel(1)
model.build(tf.TensorShape([1, None]))
model.load_weights(os.path.join(pathes['model_dir'], weight))
model.reset_states()
pred = model(initBeat)
pred = [p.numpy().squeeze() for p in pred]

music_sequence = []
for track in range(params['track_size']):
    music_sequence.append([idx2beat[track][idx] for idx in initBeat[track][0].tolist()])

outputs['length'] = 100
for i in tqdm(range(outputs['length'])):

    nextBeatList = []
    for index, p in enumerate(pred):
        topk = tf.math.top_k(p[-1], 3)
        topkChoices = topk[1].numpy().squeeze()
        topkValues = topk[0].numpy().squeeze()

        # Apply random
        # next_beat = topkChoices[0]
        next_beat = None
        if np.random.uniform(0, 1) < .5:
            next_beat = topkChoices[0]
        else:
            p_choices = tf.math.softmax(topkValues[1:]).numpy()
            next_beat = np.random.choice(topkChoices[1:], 1, p=p_choices)[0]

        music_sequence[index].append(idx2beat[index][next_beat])
        nextBeatList.append(np.array([[next_beat]]))

    pred = model.predict(nextBeatList, verbose=0)


In [ ]:
generateMidi('new100-4-2'+'.mid', music_sequence)

## Output

In [16]:
fig = plot_model(model, to_file=name+'.png', show_shapes=True, show_layer_names=True)